# Registration tools

This notebook is part of [blabla citation papier].
Before executing it, please make sure you have installed the tools properly (see installation part).

Please run the cells one by one, provide input when it is asked and press enter to validate your input, but do not modify the content of the cells.

The data structure required is the following : one main folder with the raw movies, which are 3D stacks in time. The multichannel images should be split before. The name should follow 'name_index_channel.tif' with 'name' the common part between all of your samples, 'index' the index of your sample in the dataset and 'channel' the name of the channel.

Exemple : If my acquisition includes 3 samples with 2 channels each, dapi and ecad, my folder will contain the folowing files with '70h_FGF" the common name between my samples:

________________________________________

'70h_FGF_1_dapi.tif'

'70h_FGF_1_ecad.tif'

'70h_FGF_2_dapi.tif'

'70h_FGF_2_ecad.tif'

'70h_FGF_3_dapi.tif'

'70h_FGF_3_ecad.tif'

And each file is of dimension (T,Z,X,Y)

1- Importing packages (no input required)

In [1]:
import tifffile
import os
import registrationtools
import json
import numpy as np

C:\Users\gros\Anaconda3\envs\registration-env\lib\site-packages\IO\IO.py:31: UserWarning: KLB library is not installed
  warnings.warn("KLB library is not installed")


pyklb library not found, klb files will not be generated


2) Please fill below the path to your data (ex : C:\Users\username\Desktop\DATA\20230101\) **with a slash at the end**

In [2]:
path_to_data = (input('Absolute path to your data folder:')).replace("\\", "/")

Absolute path to your data folder:C:\Users\gros\Desktop\DATA\20230207_fgf\20230214\


3) Also fill the path where you want to save the json file of your registration **with name.json file** (this might be useful in case of debugging or safety checks)
example : C:\Users\username\Desktop\Registration\json_files\temporal.json

In [3]:
path_to_json = (input('Absolute path to save the json file :')).replace("\\", "/")

Absolute path to save the json file :C:\Users\gros\Desktop\CODES\Alice_Registration\json_files\json.json


<!-- 4) To find some parts of the code, the tools need to know where in your installation is your bin folder located. If you followed the procedure from the paper, the path should look like should look like C:\Users\username\Anaconda3/envs\registration-env\Library\bin\ -->

4. Fill here the common name of your files (without the number of the sample and the channel). For example, if I have '70h_FGF_1_dapi.tif','70h_FGF_1_ecad.tif','70h_FGF_1_dapi.tif', here the common name is '70h_FGF".

In [6]:
name = input('Common name of your movies :')

Common name of your movies :H2B_FGF_100h


5. Give here the number of samples that you want to register. We suppose that these samples are acquired with the same channels and have the same name pattern.
Then, give the different numbers that are going to be registered (could be 1, 2, 3, or 1, 3, 7, no need to be ordered or adjacent)

In [7]:
number_samples = int(input('Number of samples :'))

Number of samples :1


In [ ]:
samples=[]
for i in range(number_samples) :
    s = int(input('Index of sample n°'+str(i+1) +':'))
    samples.append(s)
print('samples =',samples)

6. Give here the number of channels that you want to register

In [ ]:
number_channels = int(input('Number of channels :'))

7. If the image has multiple channels, you will need one channel who will be the reference to register the other ones. The most 'stable' one, expressed everywhere, is generally a good option.

In [ ]:
if number_channels == 1:
    print('There is only one channel. This channel will be the reference.')
else :  
    ch_ref = input('Name of the reference channel :')

In [ ]:
channels=[]
if number_channels == 1:
    ch = input('Name of the channel : ')
    channels.append(ch)
    ch_ref=ch
else : 
    print('Name of channel n°1 :', ch_ref)
    channels.append(ch_ref)
    for n in range(1,number_channels) :
        ch = input('Name of channel n°'+str(n+1) +' : ')
        channels.append(ch)

8. Please also give the voxel size of your image in XYZ, for example [0.2,0.2,1]. This can be found in Fiji, Image>Show Info.

In [ ]:
print('Voxel size of your original image (XYZ) :')
x= float(input('X :'))
y= float(input('Y :'))
z= float(input('Z :'))
ref_voxel = [x,y,z]
print('initial voxel size',ref_voxel)

9. If for any reason you prefer having an other voxel size for the registered image (for example isotropic, meaning the same voxel size in every dimension), you can tune it below. You can also keep the same size.

In [13]:
change_voxel_size = int(input('Do you want to change the voxel size of your movies ? (1 for yes, 0 for no) : '))
if change_voxel_size==1 :
    print('\nVoxel size of your image after transformation (XYZ) (ex :[1,1,1]) :')
    x= float(input('X :'))
    y= float(input('Y :'))
    z= float(input('Z :'))
    voxel_size_output = [x,y,z]
elif change_voxel_size==0 :
    voxel_size_output = ref_voxel
print('\nVoxel size after transformation',voxel_size_output)

Do you want to change the voxel size of your movies ? (1 for yes, 0 for no) : 0

Voxel size after transformation [0.62, 0.62, 2.0]


In [14]:
channels

['sulfo', 'h2b']

9. Below, we create a folder structure and cut your movie into timepoints

In [ ]:
for i in samples :
    for c in channels:
        movie = tifffile.imread(rf'{path_to_data}{name}_{i}_{c}.tif') ##if you want to change the naming of your files
        #movie = tifffile.imread(rf'{path_to_data}rigid2D/{name}{i}_{c}_rigid2D.tif')
        print('cuting movie ',name,i,'_',c,'.tif into ', movie.shape[0], ' timepoints, saving them in folder ',i,'_',c)
        directory = str(i)+"_"+c
        #creates data structure : one directory per channel and per sample
        os.mkdir(os.path.join(path_to_data,directory))
        os.mkdir(os.path.join(path_to_data+directory,"trsf"))
        os.mkdir(os.path.join(path_to_data+directory,"proj"))
        os.mkdir(os.path.join(path_to_data+directory,"stackseq"))
        ## from movie to image sequence
        for t in range(movie.shape[0]) :
            stack = movie[t,:,:,:]
            tifffile.imwrite(path_to_data+directory+"/stackseq/movie_t"+str(format(t,'03d')+'.tif'),stack)

10. The actual registration happens in the following part. The process can take few minutes depending on the size of your data. If there is an error, it will be printed either below or in the terminal window. To solve it, please consider the 'Troubleshooting' section.

In [ ]:
for i in samples :
    for c in channels:
        directory = str(i)+"_"+c
        
        movie = tifffile.imread(rf'{path_to_data}{name}_{i}_{c}.tif') ##if you want to change the naming of your files
        ref_timepoint = int(movie.shape[0]/2)
        if c==ch_ref:
            compute_transf = 1
        else :
            compute_transf = 0
        apply_trsf=1
        # # # # ##JSON FILE CONFIG
        data = {
          "path_to_data": path_to_data+directory+"/stackseq/",
          "file_name": "movie_t{t:03d}.tif",
          "trsf_folder": rf'{path_to_data}{str(i)}_{ch_ref}/trsf/',
          "output_format": path_to_data+directory+"/proj/",
          "projection_path": path_to_data+directory+"/proj/",
          "check_TP": 0,
          "voxel_size": ref_voxel,
          "voxel_size_out" : voxel_size_output,
          "first": 0,
          "last": movie.shape[0]-1,
          "not_to_do": [],
          "compute_trsf": compute_transf,
          "ref_TP": ref_timepoint,
          "trsf_type": trsf_type,
          "padding": 1,
          "recompute": 1,

          "apply_trsf":apply_trsf,
          "out_bdv": "",
          "plot_trsf":0

        }

        json_string=json.dumps(data)
        with open(path_to_json,'w') as outfile :
            outfile.write(json_string)

        tr = registrationtools.TimeRegistration(data)
        tr.run_trsf()

11. The previous cell saved an image sequence. Here we save the output as a movie.

In [ ]:
for i in samples :
    for c in channels:
        directory = str(i)+"_"+c
        stack0 =tifffile.imread(rf"{path_to_data}/{directory}/proj/movie_t000.tif")
        movie = tifffile.imread(rf'{path_to_data}{name}_{i}_{c}.tif') ##if you want to change the naming of your files
        registered_movie = np.zeros((movie.shape[0],stack0.shape[0],stack0.shape[1],stack0.shape[2]),dtype=np.float16) #one movie per channel, of format (t,z,y,x).Datatype uint16 or float34 is necessary to export as hyperstack
        for t in range(movie.shape[0]) :
            stack =tifffile.imread(rf"{path_to_data}/{directory}/proj/movie_t{format(t,'03d')}.tif")
            #we take each stack in a given timepoint
            registered_movie[t,:,:,:]=stack #and put it in a new hyperstack
        tifffile.imwrite(path_to_data+rf"/{name}_{i}_{c}_rigid2D.tif",registered_movie.astype(np.float16)) #write a hyperstack in the main folder
        print(registered_movie.shape)